# G only KEGG based network architecture

> 

In [1]:
import numpy as np
import pandas as pd

from EnvDL.core import ensure_dir_path_exists 
from EnvDL.dlfn import g2fc_datawrapper, BigDataset, plDNN_general
from EnvDL.dlfn import ResNet2d, BasicBlock2d
from EnvDL.dlfn import LSUV_

import torch
import torch.nn.functional as F # F.mse_loss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger

from EnvDL.dlfn import kegg_connections_build, kegg_connections_clean, kegg_connections_append_y_hat, kegg_connections_sanitize_names
from EnvDL.dlfn import VNNHelper, VisableNeuralNetwork, Linear_block_reps
from EnvDL.dlfn import plDNN_general, BigDataset
from EnvDL.dlfn import reverse_edge_dict, reverse_node_props
from EnvDL.dlfn import VNNVAEHelper, plVNNVAE
from EnvDL.dlfn import kegg_connections_build, kegg_connections_clean, kegg_connections_append_y_hat, kegg_connections_sanitize_names
from EnvDL.dlfn import VNNHelper, VisableNeuralNetwork, Linear_block_reps
from EnvDL.dlfn import ListDataset, plVNN
from EnvDL.dlfn import plDNN_general, BigDataset

In [2]:
import plotly.express as px

In [3]:
# isinstance(model[0], nn.Linear)

In [4]:
cache_path = '../nbs_artifacts/02.40_g2fc_G_ACGT_VNN_baseline/'
save_prefix = [e for e in cache_path.split('/') if e != ''][-1]

# Run settings: 
max_epoch  = 20
batch_size = 48
batch_size = 4

# VNN settings:
default_out_nodes_inp   = 2 #  4
default_out_nodes_edge  = 1 # 32
default_out_nodes_out   = 1

default_drop_nodes_inp  = 0.0
default_drop_nodes_edge = 0.
default_drop_nodes_out  = 0.0

default_reps_nodes_inp  = 1
default_reps_nodes_edge = 1
default_reps_nodes_out  = 1

In [5]:
use_gpu_num = 0

device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

Using cuda device


In [6]:
ensure_dir_path_exists(dir_path = cache_path)

In [7]:
from EnvDL.dlfn import kegg_connections_build, kegg_connections_clean, kegg_connections_append_y_hat, kegg_connections_sanitize_names
from EnvDL.dlfn import VNNHelper, VisableNeuralNetwork, Linear_block_reps
from EnvDL.dlfn import ListDataset, plVNN

## Fit Using VNNHelper

In [8]:

# Same setup as above to create kegg_gene_brite
X = g2fc_datawrapper()
X.set_split()
X.load_all(name_list = ['obs_geno_lookup', 'YMat', 'KEGG_slices',], store=True) 
X.calc_cs('YMat', version = 'np', filter = 'val:train')
ACGT_gene_slice_list =     X.get('KEGG_slices', ops_string='')
parsed_kegg_gene_entries = X.get('KEGG_entries')


# Restrict to only those with pathway
kegg_gene_brite = [e for e in parsed_kegg_gene_entries if 'BRITE' in e.keys()]

# also require to have a non-empty path
kegg_gene_brite = [e for e in kegg_gene_brite if not e['BRITE']['BRITE_PATHS'] == []]

print('Retaining '+ str(round(len(kegg_gene_brite)/len(parsed_kegg_gene_entries), 4)*100)+'%, '+str(len(kegg_gene_brite)
    )+'/'+str(len(parsed_kegg_gene_entries)
    )+' Entries'
    )
# kegg_gene_brite[1]['BRITE']['BRITE_PATHS']

Loading and storing default `phno`.


Retaining 43.53%, 6067/13939 Entries


In [9]:
kegg_connections = kegg_connections_build(kegg_gene_brite = kegg_gene_brite, 
                                          n_genes = 6067) #FIXME
                                          # n_genes = 2) 
kegg_connections = kegg_connections_clean(         kegg_connections = kegg_connections)
kegg_connections = kegg_connections_append_y_hat(  kegg_connections = kegg_connections)
kegg_connections = kegg_connections_sanitize_names(kegg_connections = kegg_connections, 
                                                   replace_chars = {'.':'_'})

100%|██████████| 6067/6067 [00:00<00:00, 58976.26it/s]

Removed node "Others"


In [10]:
import re

def name_cleanup(input = '7_1_2_1P-TypeH+-ExportingTransporter', newline_char_threshold = 10):
    inp = input
    # remove "7_1_2_1" type from front of name
    rm_front = re.match(r'^[\d|_]+', inp)
    if rm_front:
        inp = inp[rm_front.span()[1]:]

    word_splits = [e.span()[0] for e in re.finditer('[a-z][A-Z]', inp)]

    word_list = []
    i = 0
    for jth in range(len(word_splits)):
        j = word_splits[jth]
        j += 1
        word_list += [inp[i:j]]
        i = j

        if jth+1 == len(word_splits):
            word_list += [inp[i:len(inp)]]

    x = []
    n = 0
    for e in word_list:
        n += len(e)
        if n >= newline_char_threshold:
            x += ['\n'+e]
            n = len(e)
        else:
            x += [' '+e]
    x = ''.join(x).strip('^ ')

    # if the name was only numerics keep the name as is
    if x != '':
        pass
    elif inp != '':
        x = inp
    elif inp == '':
        x = input

    return(x)

name_cleanup(input = '987987897', newline_char_threshold = 10)

'987987897'

In [11]:
# from graphviz import Digraph

# # kegg_connections = {
# #  'A': ['100278565'],
# #  'B': ['100278565'],
# #  'C': ['100383860'],
# #  'D': ['B', 'C'],
# #  'y_hat': ['A', 'D']}

# dot = Digraph()
# for key in kegg_connections.keys():
#     key_label = name_cleanup(input = key, newline_char_threshold = 20)+'\n '
#     dot.node(key, key_label)
#     for value in kegg_connections[key]:
#         # edge takes a head/tail whereas edges takes name pairs concatednated (A, B -> AB)in a list
#         dot.edge(value, key)    

# dot

In [12]:
# initialize helper for input nodes
myvnn = VNNHelper(edge_dict = kegg_connections)

myvnn.nodes_inp[0:10]

# Get a mapping of brite names to tensor list index
find_names = myvnn.nodes_inp # e.g. ['100383860', '100278565', ... ]
lookup_dict = {}

# the only difference lookup_dict and brite_node_to_list_idx_dict above is that this is made using the full set of genes in the list 
# whereas that is made using kegg_gene_brite which is a subset
for i in range(len(parsed_kegg_gene_entries)):
    if 'BRITE' not in parsed_kegg_gene_entries[i].keys():
        pass
    elif parsed_kegg_gene_entries[i]['BRITE']['BRITE_PATHS'] == []:
        pass
    else:
        name = parsed_kegg_gene_entries[i]['BRITE']['BRITE_PATHS'][0][-1]
        if name in find_names:
            lookup_dict[name] = i
lookup_dict    

{'100278565': 0,
 '100383860': 1,
 '100383837': 3,
 '100191673': 8,
 '100275685': 9,
 '103630585': 13,
 '100194370': 16,
 '100194192': 17,
 '100273289': 18,
 '100037826': 19,
 '100192899': 21,
 '100304252': 26,
 '100280063': 28,
 '103630746': 30,
 '100282726': 31,
 '100285519': 33,
 '100272424': 35,
 '100381824': 36,
 '100277985': 40,
 '100283343': 42,
 '100284607': 45,
 '100191772': 48,
 '100277385': 49,
 '100381530': 50,
 '103630860': 51,
 '100384769': 52,
 '103630917': 55,
 '100194239': 56,
 '103644387': 57,
 '103631010': 58,
 '103631056': 63,
 '100191905': 64,
 '100281199': 68,
 '100283731': 70,
 '103631177': 72,
 '100281329': 76,
 '100274981': 78,
 '100382167': 86,
 '100383301': 87,
 '100285254': 90,
 '100280817': 91,
 '542672': 92,
 '542290': 93,
 '103631462': 94,
 '100283425': 96,
 '103631568': 97,
 '100280768': 98,
 '103631596': 102,
 '542230': 105,
 '103631647': 106,
 '100383383': 107,
 '100274307': 111,
 '100273485': 112,
 '100191592': 113,
 '100273170': 114,
 '100384051': 11

In [13]:
# # if permuting gene identities
# torch.manual_seed(5461)

# keys = [e for e in lookup_dict.keys()]

# # vals = [lookup_dict[e] for e in lookup_dict.keys()]
# # dict(zip(keys, [int(i) for i in torch.randperm(len(keys))]))

# idx = torch.tensor([lookup_dict[e] for e in myvnn.nodes_inp])
# idx = idx[torch.randperm(idx.shape[0])]
# idx = [int(i) for i in idx]
# temp = dict(zip(myvnn.nodes_inp, idx))

# randomized_lookup_dict = {}
# for e in lookup_dict.keys():
#     if e not in temp.keys():
#         randomized_lookup_dict[e] = lookup_dict[e]
#     else:
#         randomized_lookup_dict[e] = temp[e]

# lookup_dict = randomized_lookup_dict

In [14]:
brite_node_to_list_idx_dict = {}
for i in range(len(kegg_gene_brite)):
    brite_node_to_list_idx_dict[str(kegg_gene_brite[i]['BRITE']['BRITE_PATHS'][0][-1])] = i        

# Get the input sizes for the graph
size_in_zip = zip(myvnn.nodes_inp, [np.prod(ACGT_gene_slice_list[lookup_dict[e]].shape[1:]) for e  in myvnn.nodes_inp])


In [15]:

# init input node sizes
myvnn.set_node_props(key = 'inp', node_val_zip = size_in_zip)

# init node output sizes
myvnn.set_node_props(key = 'out', node_val_zip = zip(myvnn.nodes_inp, [default_out_nodes_inp  for e in myvnn.nodes_inp]))
myvnn.set_node_props(key = 'out', node_val_zip = zip(myvnn.nodes_edge,[default_out_nodes_edge for e in myvnn.nodes_edge]))
myvnn.set_node_props(key = 'out', node_val_zip = zip(myvnn.nodes_out, [default_out_nodes_out  for e in myvnn.nodes_out]))


# # options should be controlled by node_props
myvnn.set_node_props(key = 'flatten', node_val_zip = zip(
    myvnn.nodes_inp, 
    [True for e in myvnn.nodes_inp]))

myvnn.set_node_props(key = 'reps', node_val_zip = zip(myvnn.nodes_inp, [default_reps_nodes_inp  for e in myvnn.nodes_inp]))
myvnn.set_node_props(key = 'reps', node_val_zip = zip(myvnn.nodes_edge,[default_reps_nodes_edge for e in myvnn.nodes_edge]))
myvnn.set_node_props(key = 'reps', node_val_zip = zip(myvnn.nodes_out, [default_reps_nodes_out  for e in myvnn.nodes_out]))

myvnn.set_node_props(key = 'drop', node_val_zip = zip(myvnn.nodes_inp, [default_drop_nodes_inp  for e in myvnn.nodes_inp]))
myvnn.set_node_props(key = 'drop', node_val_zip = zip(myvnn.nodes_edge,[default_drop_nodes_edge for e in myvnn.nodes_edge]))
myvnn.set_node_props(key = 'drop', node_val_zip = zip(myvnn.nodes_out, [default_drop_nodes_out  for e in myvnn.nodes_out]))

# init edge node input size (propagate forward input/edge outpus)
myvnn.calc_edge_inp()

# myvnn.mk_digraph(include = ['node_name', 'inp_size', 'out_size'])
# myvnn.mk_digraph(include = [''])

In [16]:
from EnvDL.dlfn import plDNN_general, BigDataset

In [17]:
vals = X.get('KEGG_slices', ops_string='asarray from_numpy float')

In [18]:
# restrict to the tensors that will be used
vals = [vals[lookup_dict[i]] for i in myvnn.nodes_inp] #TODO CONFIRM.
# send to gpu
# vals = [val.to('cuda') for val in vals]

In [19]:
# replace lookup so that it matches the lenght of the input tensors
new_lookup_dict = {}
for i in range(len(myvnn.nodes_inp)):
    new_lookup_dict[myvnn.nodes_inp[i]] = i
    # print((myvnn.nodes_inp[i], i))
    # break

In [20]:
## start insert

### Calculate nodes membership in each matrix and positions within each

In [21]:
# myvnn.nodes_inp
# vals[new_lookup_dict['100282167']].shape

In [22]:
node_props = myvnn.node_props
# Linear_block = Linear_block_reps,
edge_dict = myvnn.edge_dict
dependancy_order = myvnn.dependancy_order
node_to_inp_num_dict = new_lookup_dict

In [23]:
# check dep order

tally = []
for d in dependancy_order:
    if edge_dict[d] == []:
        tally.append(d)
    elif False not in [True if e in tally else False for e in edge_dict[d]]:
        tally.append(d)
    else:
        print('error!')
        break

In [24]:
# build output nodes 
d_out = {0:[]}
for d in dependancy_order:
    if edge_dict[d] == []:
        d_out[min(d_out.keys())].append(d)
    else:
        # print((d, edge_dict[d]))

        d_out_i = 1+max(sum([[key for key in d_out.keys() if e in d_out[key]]
                   for e in edge_dict[d]], []))
        
        if d_out_i not in d_out.keys():
            d_out[d_out_i] = []
        d_out[d_out_i].append(d)

In [25]:
# build input nodes NOPE. THE PASSHTROUGHS! 
d_eye = {}
tally = []
for i in range(max(d_out.keys()), min(d_out.keys()), -1):
    # print(i)
    nodes_needed = sum([edge_dict[e] for e in d_out[i]], [])+tally
    # check against what is there and then dedupe
    nodes_needed = [e for e in nodes_needed if e not in d_out[i-1]]
    nodes_needed = list(set(nodes_needed))
    tally = nodes_needed
    d_eye[i] = nodes_needed

# d_inp[0]= d_out[0]
    
[len(d_eye[i]) for i in d_eye.keys()]

[58, 55, 106, 89, 105, 43, 0]

In [26]:
[(key, len(d_out[key])) for key in d_out.keys()]

[(0, 6067), (1, 2167), (2, 447), (3, 143), (4, 25), (5, 8), (6, 10), (7, 1)]

In [27]:
dd = {}
for i in d_eye.keys():
    dd[i] = {'out': d_out[i],
             'inp': d_out[i-1],
             'eye': d_eye[i]}
# plus special 0 layer that handles the snps
    
dd[0] = {'out': d_out[0],
         'inp': d_out[0],
         'eye': []}

In [28]:
# check that the output nodes' inputs are satisfied by the same layer's inputs (inp and eye)

for i in dd.keys():
    # out node in each
    for e in dd[i]['out']:
        # node depends in inp/eye
        node_pass_list = [True if ee in dd[i]['inp']+dd[i]['eye'] else False 
                          for ee in edge_dict[e]]
        if False not in node_pass_list:
            pass
        else:
            print('exit') 

In [29]:
print("Layer\t#In\t#Out")
for i in range(min(dd.keys()), max(dd.keys())+1, 1):
    node_in      = [node_props[e]['out'] for e in dd[i]['inp']+dd[i  ]['eye'] ]
    if i == max(dd.keys()):
        node_out = [node_props[e]['out'] for e in dd[i]['out'] ]
    else:
        node_out = [node_props[e]['out'] for e in dd[i]['out']+dd[i+1]['eye']]
    print(f'{i}:\t{sum(node_in)}\t{sum(node_out)}')

Layer	#In	#Out
0:	12134	12134
1:	12134	2253
2:	2253	552
3:	552	232
4:	232	131
5:	131	63
6:	63	68
7:	68	1


### Creating Structured Matrices for Layers

In [30]:
dd.keys()

dict_keys([7, 6, 5, 4, 3, 2, 1, 0])

In [31]:
class structured_layer_info:
    def __init__(self, i, 
                 dd,  # {1: {'out': ['OtherTubulinModificationProteins',
                      #      'inp': [
                      #      'eye': [
                 node_props, # {'KeggOrthology(Ko)[Br-Zma00001]': {'out': 1, 'reps': 1, 'drop': 0.0, 'inp': 7},
                 edge_dict
                 ):
        self.row_inp = dd[i]['inp']
        self.row_eye = dd[i]['eye']

        self.col_out = dd[i]['out']
        self.col_eye = []
        if i+1 in dd.keys():
            self.col_eye = dd[i+1]['eye'] 

        # build lookup dicts of the information on each side
        row_nodes = [e for e in self.row_inp+self.row_eye]
        col_nodes = [e for e in self.col_out+self.col_eye]

        if i == min(dd.keys()):
            # print('check')
            row_sizes = [node_props[e]['inp'] for e in row_nodes]
        else:
            row_sizes = [node_props[e]['out'] for e in row_nodes]
        col_sizes = [node_props[e]['out'] for e in col_nodes]

        row_sizes = torch.Tensor(row_sizes).to(torch.int)
        row_stop  = torch.cumsum(row_sizes, 0)
        row_start = torch.concat([torch.Tensor([0]).to(torch.int), row_stop[0:-1]])

        col_sizes = torch.Tensor(col_sizes).to(torch.int)
        col_stop  = torch.cumsum(col_sizes, 0)
        col_start = torch.concat([torch.Tensor([0]).to(torch.int), col_stop[0:-1]])

        self.row_info = {}
        for j in range(len(row_sizes)):
            self.row_info[row_nodes[j]]= {
                # 'row_nodes': row_nodes[j],
                'size': row_sizes[j],
                     'stop':  row_stop[j],
                    'start': row_start[j],
            }

        self.col_info = {}
        for j in range(len(col_sizes)):
            self.col_info[col_nodes[j]]= {
                # 'col_nodes': col_nodes[j],
                'size': col_sizes[j],
                     'stop':  col_stop[j],
                    'start': col_start[j],
            }
    
        ## Init weight & bias matrix ====
        self.weight          = torch.zeros([row_stop[-1], col_stop[-1]])
        self.weight_bool     = torch.zeros([row_stop[-1], col_stop[-1]]) # 1 if is weight
        self.weight_eye_bool = torch.zeros([row_stop[-1], col_stop[-1]]) # 1 if is eye
        self.bias            = torch.zeros([              col_stop[-1]])
        self.bias_eye_bool   = torch.zeros([              col_stop[-1]]) # 1 if is eye

        for e in self.col_out:
            c_size = self.col_info[e]['size']
            # print(f'i {i} key min {min(dd.keys())}')
            if i == min(dd.keys()):
                inps = [e]
            else:
                inps = edge_dict[e]
            # print(f'inps: {inps}')
            # r_size_total = sum([self.row_info[ee]['size'] for ee in inps])
            # W = torch.empty(r_size_total, c_size)
            # W = torch.nn.init.kaiming_normal_(W, a=0, mode='fan_in', nonlinearity='relu')
        
            c1 = self.col_info[e]['start']
            c2 = self.col_info[e]['stop']

            # W_start = 0
            # print(W.shape)
            for inp in inps:
                r1 = self.row_info[inp]['start']
                r2 = self.row_info[inp]['stop']
                slice_size = r2-r1
                # W_end = W_start + slice_size
                # print(W_start, W_end)
                # self.weight[r1:r2, c1:c2] = W[W_start:W_end]

                # Use nn.Linear to initialize the matrix instead of doing it manually.
                xx = nn.Linear(slice_size, c_size)
                W = xx.weight.clone().detach().requires_grad_(False)
                # print(f'{W.shape} {self.weight[r1:r2, c1:c2].shape}')
                B = xx.bias.clone().detach().requires_grad_(False)
                self.weight[r1:r2, c1:c2] = W.swapaxes(0,1)                                                          # <- transposed to match nn.Linear
                self.weight_bool[r1:r2, c1:c2] = torch.ones(W.shape).swapaxes(0,1) # Fill in gradient bool matrix    # <- transposed to match nn.Linear
                self.bias[c1:c2] = B
                # W_start = W_end
                
        for e in self.col_eye:
            c_size = self.col_info[e]['size']
            c1 = self.col_info[e]['start']
            c2 = self.col_info[e]['stop']
            r1 = self.row_info[e]['start']
            r2 = self.row_info[e]['stop']

            W = torch.eye(c_size)
            self.weight[r1:r2, c1:c2] = W
            # FIXME testing if not allowing gradients on unity entries is causing the problem. If it is then either 
            # 1. pass through gradients from one layer to the next (and or)
            # 2. re-set these values to unity after each update. 
            self.weight_eye_bool[r1:r2, c1:c2] = torch.eye(c_size).swapaxes(0,1)                                     # <- transposed to match nn.Linear

        ## Init identity components of matrix ====
        # 1.0 if identity otherwise 0
        for e in self.col_eye:
            self.bias_eye_bool[self.col_info[e]['start']:self.col_info[e]['stop']] = 1.0
        if self.col_eye != []:
            self.bias_eye_bool = self.bias_eye_bool        

# i = 0
# M =structured_layer_info(i, dd, node_props, edge_dict)
# px.imshow(M.weight.swapaxes(0,1))

In [32]:
M_list = [structured_layer_info(i = ii, dd = dd, node_props= node_props, edge_dict = edge_dict) for ii in range(0, max(dd.keys())+1)]


In [33]:
[e.weight.shape for e in M_list]

[torch.Size([144468, 12134]),
 torch.Size([12134, 2253]),
 torch.Size([2253, 552]),
 torch.Size([552, 232]),
 torch.Size([232, 131]),
 torch.Size([131, 63]),
 torch.Size([63, 68]),
 torch.Size([68, 1])]

### Setup Dataloader using `M_list`

In [34]:
vals = X.get('KEGG_slices', ops_string='asarray from_numpy float')
# restrict to the tensors that will be used
vals = torch.concat([vals[lookup_dict[i]].reshape(4926, -1) 
                     for i in M_list[0].row_inp
                    #  for i in dd[0]['inp'] # matches
                     ], axis = 1)
vals.shape
vals = vals.to('cuda')

In [35]:
training_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = True,
    lookup_obs =  X.get('val:train',       ops_string='                   asarray from_numpy'), 
    lookup_geno = X.get('obs_geno_lookup', ops_string='   filter:val:train asarray from_numpy'),
    y =           X.get('YMat',            ops_string='cs filter:val:train asarray from_numpy float cuda:0')[:, None],
    # y =           X.get('YMat',            ops_string='cs filter:val:train asarray from_numpy float')[:, None],
    G =           vals,
    G_type = 'raw',
    # send_batch_to_gpu = 'cuda:0'
    ),
    batch_size = batch_size,
    shuffle = True
)

# validation_dataloader = DataLoader(BigDataset(
#     lookups_are_filtered = True,
#     lookup_obs =  X.get('val:test',        ops_string='                   asarray from_numpy'), 
#     lookup_geno = X.get('obs_geno_lookup', ops_string='   filter:val:test asarray from_numpy'),
#     y =           X.get('YMat',            ops_string='cs filter:val:test asarray from_numpy float cuda:0')[:, None],
#     G =           vals,
#     G_type = 'raw',
#     # send_batch_to_gpu = 'cuda:0'
#     ),
#     batch_size = batch_size,
#     shuffle = False
# )


## Structured Layer

In [36]:
# px.imshow(M.weight.swapaxes(0,1))

In [37]:
# xx = nn.Linear(M.weight.shape[0], M.weight.shape[1])

# xx.weight.requires_grad = False

In [38]:
# px.imshow(xx.weight)

In [39]:
# xx.weight = torch.nn.Parameter(M.weight.swapaxes(0,1))
# xx.weight.requires_grad = True
# px.imshow(xx.weight.detach())

In [40]:
layer_list = []
for i in range(len(M_list)):
    l = nn.Linear(M_list[i].weight.shape[0], M_list[i].weight.shape[1])
    l.weight.requires_grad = False
    l.weight = torch.nn.Parameter(M_list[i].weight.swapaxes(0,1))
    l.weight.requires_grad = True

    l.bias.requires_grad = False
    l.bias = torch.nn.Parameter(M_list[i].bias)
    l.bias.requires_grad = True

    layer_list += [l]
    
    if i+1 != len(M_list):
        layer_list += [nn.ReLU()]


In [41]:
class NeuralNetwork(nn.Module):
    def __init__(self, layer_list):
        super(NeuralNetwork, self).__init__()
        self.layer_list = nn.ModuleList(layer_list)
        
    def forward(self, x):
        for l in self.layer_list:
            x = l(x)
        return x

model = NeuralNetwork(layer_list)

In [42]:
model = model.to('cuda')

In [44]:
y_i, xs_i = next(iter(training_dataloader))

In [45]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [46]:
from tqdm import tqdm

In [47]:
for ii in tqdm(range(2)):
    pred = model(xs_i)
    loss = loss_fn(pred, y_i)
    if ii % 100 == 0:
        print(f'{loss}')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  0%|          | 0/2 [00:00<?, ?it/s]


2.945368766784668


OutOfMemoryError: CUDA out of memory. Tried to allocate 6.53 GiB. GPU 0 has a total capacty of 23.65 GiB of which 699.06 MiB is free. Including non-PyTorch memory, this process has 22.95 GiB memory in use. Of the allocated memory 22.48 GiB is allocated by PyTorch, and 16.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# After training are unity weights still unity?
i = 2
(
M_list[i].bias_eye_bool, 
M_list[i].weight_eye_bool,
model.layer_list[((2*(i)))].weight,
model.layer_list[((2*(i)))].weight.grad
)

In [ ]:
dd

In [ ]:
M.row_inp
M.row_eye
M.col_out
M.col_eye

M.row_info
# M.col_info

# M.weight
# M.weight_bool
# M.weight_eye_bool
# M.bias
# M.bias_eye_bool

In [ ]:
query = list(M_list[i].col_info.keys())[0]
slice_accumulator = []
# M_list[i].row_info[query]
c1 = M_list[i].col_info[query]['start'] 
c2 = M_list[i].col_info[query]['stop']

# could get full slice then drop zero values too
for e in edge_dict[query]:
    r1 = M_list[i].row_info[e]['start'] 
    r2 = M_list[i].row_info[e]['stop']
    slice_accumulator += [model.layer_list[((2*(i)))].weight.swapaxes(0,1)[r1:r2, c1:c2].clone().detach().requires_grad_(False)]

In [ ]:
torch.concat(slice_accumulator)

In [ ]:
model.layer_list[((2*(i)))].weight

In [ ]:
dot

In [ ]:
node_to_weights = {}
for i in range(0, len(M_list)):
    for query in M_list[i].col_out:
        slice_accumulator = []
        # M_list[i].row_info[query]
        c1 = M_list[i].col_info[query]['start'] 
        c2 = M_list[i].col_info[query]['stop']

        # could get full slice then drop zero values too
        if i == 0:
            r1 = M_list[i].row_info[query]['start'] 
            r2 = M_list[i].row_info[query]['stop']
            slice_accumulator += [model.layer_list[((2*(i)))].weight.swapaxes(0,1)[r1:r2, c1:c2].clone().detach().requires_grad_(False)]

        else: 
            for e in edge_dict[query]:
                r1 = M_list[i].row_info[e]['start'] 
                r2 = M_list[i].row_info[e]['stop']
                slice_accumulator += [model.layer_list[((2*(i)))].weight.swapaxes(0,1)[r1:r2, c1:c2].clone().detach().requires_grad_(False)]

        slice_accumulator = torch.concat(slice_accumulator)
        node_to_weights[query] = slice_accumulator

In [ ]:
xx = [np.round(float(node_to_weights[key].abs().mean()), 3) for key in node_to_weights.keys()]
color_vals = ['#ffffff', '#fff7ec', '#fee8c8', '#fdd49e', '#fdbb84', '#fc8d59', '#ef6548', '#d7301f', '#b30000'#, '#7f0000'
              ]
color_cutoffs = [i*max(xx)/len(color_vals) for i in range(len(color_vals))]


dot = Digraph()
for key in node_to_weights.keys():


    key_mean_w = np.round(float(node_to_weights[key].abs().mean()), 3)
    color_val = color_vals[[i for i in range(len(color_cutoffs)) if key_mean_w >= color_cutoffs[i]][-1]]

    # key_label = name_cleanup(input = key, newline_char_threshold = 20)+'\nMean: '+str(key_mean_w)
    # dot.node(key, key_label, style='filled', fillcolor=color_val)  
    
    
    key_label = name_cleanup(input = key, newline_char_threshold = 20)+'\n           '  
    dot.node(key, key_label)

    if key in kegg_connections.keys():
        for value in kegg_connections[key]:
            # edge takes a head/tail whereas edges takes name pairs concatednated (A, B -> AB)in a list
            dot.edge(value, key)    

dot

In [ ]:
## end insert

In [ ]:
# model = VisableNeuralNetwork(
#     node_props = myvnn.node_props,
#     Linear_block = Linear_block_reps,
#     edge_dict = myvnn.edge_dict,
#     dependancy_order = myvnn.dependancy_order,
#     node_to_inp_num_dict = new_lookup_dict
# )
# model = model.to('cuda')
# # # with torch.no_grad(): print(model(vals))

In [ ]:
# # if randomizing y
# torch.manual_seed(2608434)

# y_trn = X.get('YMat', ops_string='cs filter:val:train asarray from_numpy float')
# y_trn = y_trn[torch.randperm(y_trn.shape[0])]


# y_val = X.get('YMat', ops_string='cs filter:val:train asarray from_numpy float')
# y_val = y_val[torch.randperm(y_val.shape[0])]


In [ ]:

# training_dataloader = DataLoader(BigDataset(
#     lookups_are_filtered = True,
#     lookup_obs =  X.get('val:train',       ops_string='                   asarray from_numpy'), 
#     lookup_geno = X.get('obs_geno_lookup', ops_string='   filter:val:train asarray from_numpy'),
#     y =           X.get('YMat',            ops_string='cs filter:val:train asarray from_numpy float cuda:0')[:, None],
#     G =           vals,
#     G_type = 'list',
#     # send_batch_to_gpu = 'cuda:0'
#     ),
#     batch_size = batch_size,
#     shuffle = True
# )

# validation_dataloader = DataLoader(BigDataset(
#     lookups_are_filtered = True,
#     lookup_obs =  X.get('val:test',        ops_string='                   asarray from_numpy'), 
#     lookup_geno = X.get('obs_geno_lookup', ops_string='   filter:val:test asarray from_numpy'),
#     y =           X.get('YMat',            ops_string='cs filter:val:test asarray from_numpy float cuda:0')[:, None],
#     G =           vals,
#     G_type = 'list',
#     # send_batch_to_gpu = 'cuda:0'
#     ),
#     batch_size = batch_size,
#     shuffle = False
# )


In [ ]:
# LSUV_(model, data = next(iter(training_dataloader))[1])

In [ ]:
# VNN = plDNN_general(model)  

# optimizer = VNN.configure_optimizers()

# logger = TensorBoardLogger("tb_vnn_logs", name=save_prefix)
# trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

# trainer.fit(model=VNN, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)


In [ ]:
# import time, json
# save_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())

# json_path = cache_path+''.join(['lookup_dict','__'+save_time,'.json'])
# with open(json_path, 'w', encoding='utf-8') as f: 
#     json.dump(new_lookup_dict, f, ensure_ascii=False, indent=4)    

# pt_path = cache_path+''.join([save_prefix,'__'+save_time,'.pt'])

# torch.save(VNN.mod, pt_path)